# Convolutional SNN
### **Classifying Fashion-MNIST with Convolutional SNN**

This tutorial goes over how to train a convolutional spiking neural network (CSNN) on the Fashion-MNIST dataset and deploy on HiAER Spike using our conversion pipline.

### **Define a CSNN**
To build a CSNN with PyTorch, we can use snnTorch, SpikingJelly or other deep learning frameworks that are based on PyTorch. Currently, our conversion pipline supports snnTorch and SpikingJelly. In this tutorial, we will be using SpikingJelly.

Install the PyPi distribution of SpikingJelly

In [4]:
pip install spikingjelly


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Import necessary libraries from SpikingJelly and PyTorch

In [5]:
from spikingjelly.activation_based import neuron, functional, surrogate, layer
import torch 
import torch.nn as nn

### **Model Architecture**
Using SpikingJelly, we can define a CSNN with the architecture of 8C3-BN-6272FC10
- 8C3: a 3x3 convolutional kernel with 8 channels
- BN: batch normalization layer 
- 6272FC10: the fully connected output layer 
 
#### **Surrogate Function**
SpikingJelly and snnTorch both use backpropagation through time to train the spiking neural networks. However, because of the non-differentiability of spikes, surrogate gradients are used in place of the Heaviside function in the backward pass

In [6]:
class model(nn.Module): 
    def __init__(self, channels=8): 
        super().__init__()
        self.conv = nn.Conv2d(1, channels, kernel_size=3, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(channels)
        self.lif1 = neuron.IFNode(surrogate_function=surrogate.ATan())
        self.flat = nn.Flatten()
        self.linear = nn.Linear(channels * 28 * 28, 10, bias=False)
        self.lif2 = neuron.IFNode(surrogate_function=surrogate.ATan())
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.lif1(x)
        x = self.flat(x)
        x = self.linear(x)
        x = self.lif2(x)
        return x

In [7]:
#Initiate the Network
net = model()

### **Setting up the MNIST Dataset**

In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#Download Fashion-MNIST data from torch 
fashion_mnist_train = datasets.FashionMNIST('data/fashion_mnist', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]))
fashion_mnist_test = datasets.FashionMNIST('data/fashion_mnist', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]))

# Create DataLoaders
train_loader = DataLoader(fashion_mnist_train, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(fashion_mnist_test, batch_size=128, shuffle=True, drop_last=True)


### **Training the SNN**
Since we are using a static image dataset, we will first encode the image into spikes using the rate encoding function from spikingjelly. With rate encoding, the input feature determines the firing frequency and the neuron that fries the most is selected as the predicted class.  

In [9]:
from spikingjelly.activation_based import encoding
import time

In [10]:
#Setting up the encoder and the time steps
encoder = encoding.PoissonEncoder()
num_steps = 40

#Define training parameters
epochs = 20
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

#Copy netowrk to device 
net.to(device)

#Define optimizer, scheduler and the loss function
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
loss_fun = torch.nn.MSELoss()

In [11]:
for epoch in range(epochs):
    start_time = time.time()
    net.train()
    train_loss = 0
    train_acc = 0
    train_samples = 0
    for img, label in train_loader:
        optimizer.zero_grad()
        img = img.to(device)
        label = label.to(device)
        label_onehot = torch.nn.functional.one_hot(label, 10).float()
        out_fr = 0.
        for t in range(num_steps):
            encoded_img = encoder(img)
            out_fr += net(encoded_img)
        out_fr = out_fr/num_steps  
        loss = loss_fun(out_fr, label_onehot)
        loss.backward()
        optimizer.step()

        train_samples += label.numel()
        train_loss += loss.item() * label.numel()
        train_acc += (out_fr.argmax(1) == label).float().sum().item()

        #reset the membrane protential after each input image
        functional.reset_net(net)

    train_time = time.time()
    train_speed = train_samples / (train_time - start_time)
    train_loss /= train_samples
    train_acc /= train_samples
    
    lr_scheduler.step()
        
    net.eval()
    test_loss = 0
    test_acc = 0
    test_samples = 0

    with torch.no_grad():
        for img, label in test_loader:
            img = img.to(device)
            label = label.to(device)
            label_onehot = torch.nn.functional.one_hot(label, 10).float()
            out_fr = 0.   
            for t in range(num_steps):
                encoded_img = encoder(img)
                out_fr += net(encoded_img)
            out_fr = out_fr/num_steps  

            loss = loss_fun(out_fr, label_onehot)

            test_samples += label.numel()
            test_loss += loss.item() * label.numel()
            test_acc += (out_fr.argmax(1) == label).float().sum().item()
            functional.reset_net(net)

    test_time = time.time()
    test_speed = test_samples / (test_time - train_time)
    test_loss /= test_samples
    test_acc /= test_samples

    print(f'epoch = {epoch}, train_loss ={train_loss: .4f}, train_acc ={train_acc: .4f}, test_loss ={test_loss: .4f}, test_acc ={test_acc: .4f}')
    print(f'train speed ={train_speed: .4f} images/s, test speed ={test_speed: .4f} images/s')

epoch = 0, train_loss = 0.0357, train_acc = 0.7461, test_loss = 0.0235, test_acc = 0.8520
train speed = 2935.7557 images/s, test speed = 5597.1990 images/s
epoch = 1, train_loss = 0.0207, train_acc = 0.8726, test_loss = 0.0210, test_acc = 0.8689
train speed = 3042.4015 images/s, test speed = 7519.0884 images/s
epoch = 2, train_loss = 0.0186, train_acc = 0.8858, test_loss = 0.0196, test_acc = 0.8771
train speed = 2991.0198 images/s, test speed = 7575.3957 images/s
epoch = 3, train_loss = 0.0174, train_acc = 0.8933, test_loss = 0.0194, test_acc = 0.8791
train speed = 2984.4427 images/s, test speed = 7826.6419 images/s
epoch = 4, train_loss = 0.0166, train_acc = 0.8992, test_loss = 0.0188, test_acc = 0.8817
train speed = 2894.7927 images/s, test speed = 7270.1850 images/s
epoch = 5, train_loss = 0.0161, train_acc = 0.9024, test_loss = 0.0186, test_acc = 0.8845
train speed = 2872.4032 images/s, test speed = 7255.2524 images/s
epoch = 6, train_loss = 0.0156, train_acc = 0.9071, test_loss = 

### **Converting the trained SNN to HiAER Spike Format**

In [12]:
from hs_api.converter import CRI_Converter, Quantize_Network, BN_Folder
from hs_api.api import CRI_network
# import hs_bridge #Uncomment when running on FPGA

#Fold the BN layer 
bn = BN_Folder() 
net_bn = bn.fold(net)

#Weight, Bias Quantization 
qn = Quantize_Network() 
net_quan = qn.quantize(net_bn)

#Set the parameters for conversion
input_layer = 0 #first pytorch layer that acts as synapses
output_layer = 4 #last pytorch layer that acts as synapses
input_shape = (1, 28, 28)
backend = 'spikingjelly'
v_threshold = qn.v_threshold

cn = CRI_Converter(num_steps = num_steps, 
                   input_layer = input_layer, 
                   output_layer = output_layer, 
                   input_shape = input_shape,
                   backend=backend,
                   v_threshold = v_threshold)
cn.layer_converter(net_quan)

Quantized:  conv
Quantized:  bn
Quantized:  lif1
Quantized:  flat
Quantized:  linear
Quantized:  lif2
Quantization time: 0.0012586116790771484
Constructing Axons from Conv2d Layer
Input layer shape(infeature, outfeature): [ 1 28 28] [ 8 28 28]
Input.shape: (28, 28)
input_padded: (1, 30, 30)


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  5.38it/s]


Constructing 8 bias axons from conv layer.
Numer of neurons: 0, number of axons: 792
Converting Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) takes 0.33516740798950195
Constructing neurons from linear Layer
Hidden layer shape(infeature, outfeature):  (8, 28, 28) 10
Instantiate output neurons
Numer of neurons: 6282, number of axons: 792


### **Initiate the HiAER Spike SNN**

In [ ]:
config = {}
config['neuron_type'] = "I&F"
config['global_neuron_params'] = {}
config['global_neuron_params']['v_thr'] = int(quan_fun.v_threshold)
    
# #Uncomment this to create a network running on the FPGA
# hardwareNetwork = CRI_network(dict(cri_convert.axon_dict),
#                               connections=dict(cri_convert.neuron_dict),
#                               config=config,target='CRI', 
#                               outputs = cri_convert.output_neurons,
#                               coreID=1)

softwareNetwork = CRI_network(dict(cri_convert.axon_dict),
                              connections=dict(cri_convert.neuron_dict),
                              config=config,target='simpleSim', 
                              outputs = cri_convert.output_neurons,
                              coreID=1)

### **Deploying the SNN on HiAER Spike**

run_sw and run_hw are two helper functions for running the spiking neural network 

In [ ]:
def Run_sw(self,inputList,softwareNetwork):
    predictions = []
    total_time_cri = 0
    #each image
    for currInput in tqdm(inputList):
        #reset the membrane potential to zero
        softwareNetwork.simpleSim.initialize_sim_vars(len(self.neuron_dict))
        spikeRate = [0]*10
        #each time step
        for slice in currInput:
            start_time = time.time()
            swSpike = softwareNetwork.step(slice, membranePotential=False)

            end_time = time.time()
            total_time_cri = total_time_cri + end_time-start_time
            for spike in swSpike:
                spikeIdx = int(spike) - self.bias_start_idx 
                try: 
                    if spikeIdx >= 0: 
                        spikeRate[spikeIdx] += 1 
                except:
                    print("SpikeIdx: ", spikeIdx,"\n SpikeRate:",spikeRate)
        predictions.append(spikeRate.index(max(spikeRate)))
    print(f"Total simulation execution time: {total_time_cri:.5f} s")
    return(predictions)

In [ ]:
def run_CRI_hw(self,inputList,hardwareNetwork):
    predictions = []
    #each image
    total_time_cri = 0
    for currInput in tqdm(inputList):
        #initiate the hardware for each image
        hs_bridge.FPGA_Execution.fpga_controller.clear(len(self.neuron_dict), False, 0)  ##Num_neurons, simDump, coreOverride
        spikeRate = [0]*10
        #each time step
        for slice in tqdm(currInput):
            start_time = time.time()
            hwSpike, latency, hbmAcc = hardwareNetwork.step(slice, membranePotential=False)
            print(f'hwSpike: {hwSpike}\n. latency : {latency}\n. hbmAcc:{hbmAcc}')
            end_time = time.time()
            total_time_cri = total_time_cri + end_time-start_time
            for spike in hwSpike:
                # print(int(spike))
                spikeIdx = int(spike) - self.bias_start_idx 
                try: 
                    if spikeIdx >= 0: 
                        spikeRate[spikeIdx] += 1 
                except:
                    print("SpikeIdx: ", spikeIdx,"\n SpikeRate:",spikeRate)
        predictions.append(spikeRate.index(max(spikeRate))) 
    print(f"Total execution time CRIFPGA: {total_time_cri:.5f} s")
    return(predictions)

In [ ]:
cri_convert.bias_start_idx = int(cri_convert.output_neurons[0])
loss_fun = nn.MSELoss()
start_time = time.time()
test_loss = 0
test_acc = 0
test_samples = 0
num_batches = 0

RUN_HARDWARE = False #Set to True if running on FPGA

for img, label in tqdm(test_loader):
    cri_input = cri_convert.input_converter(img)
    output = None
    if RUN_HARDWARE:
        output = torch.tensor(run_CRI_hw(cri_input,hardwareNetwork), dtype=float)
    else:
        output = torch.tensor(run_CRI_sw(cri_input,softwareNetwork), dtype=float)
    loss = loss_fun(output, label)
    test_samples += label.numel()
    test_loss += loss.item() * label.numel()
    test_acc += (output == label).float().sum().item()
    num_batches += 1
test_time = time.time()
test_speed = test_samples / (test_time - start_time)
test_loss /= test_samples
test_acc /= test_samples

print(f'test_loss ={test_loss: .4f}, test_acc ={test_acc: .4f}')
print(f'test speed ={test_speed: .4f} images/s')